### Install and Import Dependent Libraries

In [1]:
!pip install awswrangler --quiet
!pip install pyathena --quiet
!pip install seaborn --quiet
!pip install tensorflow --quiet
!pip install gdown --quiet

In [ ]:
# Import libraries
import boto3
import sagemaker
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import awswrangler as wr
from pyathena import connect
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.session import Session
import joblib
import json
from time import gmtime, strftime
import datetime
import random

### Initialize Common Variables

In [ ]:
# initialize Sagemaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3_client = boto3.client("s3", region_name=region)

boto_session = boto3.Session(region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

# initialize sagemaker featurestore session
feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sm,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

In [ ]:
# Default S3 bucket for offline feature store
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-sepsis"

In [ ]:
# initialize our private bucket path
s3_datalake_path_csv = "s3://{}/sepsis-prediction/csv".format(bucket)
%store s3_datalake_path_csv

In [ ]:
# initialize local data path
local_data_path_csv = '/home/sagemaker-user/data'
%store local_data_path_csv

In [ ]:
# Set the datalake path to Parquet data
s3_datalake_path_parquet = "s3://{}/sepsis-prediction/parquet".format(bucket)
%store s3_datalake_path_parquet

In [ ]:

# Set S3 staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [ ]:
# Set Athena parameters
database_name = "sepsisPatientData"

In [ ]:
# Connect to the Athena staging directory
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [ ]:
# Default store sales feature store name
store_sales_feature_group_name = "sepsis-prediction-feature-group"

In [ ]:
# define default store sales feature group
store_sales_feature_group = FeatureGroup(
    name=store_sales_feature_group_name, sagemaker_session=feature_store_session
)